In [44]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['const']=np.ones((Nobs,1))
data = df[df['Market share'] != 0]

#Scale for better intepretation

data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

data.head(20)

/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_7455/1221689314.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_7455/1221689314.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
/var/folders/84/nl1t_q050914klynvps92vkc0000gn/T/ipykernel_7455/1221689314.py:10: SettingWithC

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,const
8,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,C,CH,257,1.0
9,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,C,CH,183,1.0
10,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,C,CH,177,1.0
21,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,C,CH,3,1.0
28,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,37.5,89.010141,40.2,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,37.5,80.003519,40.2,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,37.5,78.972366,40.2,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,37.5,67.303773,40.2,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,47.2,127.889611,52.2,17,Sedan,F,DE,84,1.0


# Instruments

In [46]:
fun.BLP(data, 'HP')
fun.BLP(data, 'Chargetime')
fun.BLP(data, 'Range')

# create_instrument_localdiff(data, hvilket instrument, faktor std_dev skal ganges med (1 = 150 ca.))
# instrument hedder = instrument + '_instrument_localdiff'
fun.create_instrument_localdiff(data, 'HP', 1)
fun.create_instrument_localdiff(data, 'Chargetime', 1)
fun.create_instrument_localdiff(data, 'Range', 1)
data.head(20)

/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[instrument+'_sum'] = new_columns['sum']
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[instrument+'_quadratic'] = new_columns['quadratic']
/Users/valther/Desktop/Bachelorprojekt/Bachelor/functions.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,ID,Year,Market share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,...,const,HP_sum,HP_quadratic,Chargetime_sum,Chargetime_quadratic,Range_sum,Range_quadratic,HP_instrument_localdiff,Chargetime_instrument_localdiff,Range_instrument_localdiff
8,1,2021,0.010373,Aiways,U5,40.0,28.462170,20.1,34,SUV,...,1.0,8381.6,299317.80,11308,425032,13081.9,561789.37,3811.9,3303,4242.9
9,1,2022,0.005976,Aiways,U5,40.0,31.368183,20.1,34,SUV,...,1.0,8381.6,299317.80,11308,425032,13081.9,561789.37,3811.9,3303,4242.9
10,1,2023,0.002860,Aiways,U5,40.0,26.452400,20.1,34,SUV,...,1.0,8381.6,299317.80,11308,425032,13081.9,561789.37,3811.9,3303,4242.9
21,2,2023,0.000048,Aiways,U6,40.5,36.063800,21.4,34,SUV,...,1.0,8420.5,300071.87,11376,427344,13161.4,564949.12,3825.1,3303,4145.7
28,3,2019,0.040630,Audi,e-tron,37.5,97.970447,40.2,17,SUV,...,1.0,8240.9,292449.63,11325,427055,13014.4,559558.12,4944.9,9367,5136.8
29,3,2020,0.034680,Audi,e-tron,37.5,89.010141,40.2,17,SUV,...,1.0,8240.9,292449.63,11325,427055,13014.4,559558.12,4944.9,9367,5136.8
30,3,2021,0.010494,Audi,e-tron,37.5,80.003519,40.2,17,SUV,...,1.0,8240.9,292449.63,11325,427055,13014.4,559558.12,4944.9,9367,5136.8
31,3,2022,0.017570,Audi,e-tron,37.5,78.972366,40.2,17,SUV,...,1.0,8240.9,292449.63,11325,427055,13014.4,559558.12,4944.9,9367,5136.8
32,3,2023,0.001099,Audi,e-tron,37.5,67.303773,40.2,17,SUV,...,1.0,8240.9,292449.63,11325,427055,13014.4,559558.12,4944.9,9367,5136.8
41,4,2021,0.003391,Audi,e-tron GT,47.2,127.889611,52.2,17,Sedan,...,1.0,8285.3,292355.31,11359,427633,13060.3,559905.85,4630.0,9367,5270.6


In [47]:
# Copy the dataframe
df2 = data.copy()

In [48]:
# Creating dummy for china
df2['China'] = (df2['Country'] == 'CH').astype(int)

In [49]:
# Take the log of the market share
df2['log_market_share'] = np.log(df2['Market share'])

# BLP instrument (sum)

In [50]:
y = df2['log_market_share']
x = df2[['const', 'Range', 'HP', 'Chargetime']]
dummies = df2[[ 'China']]
X = pd.concat([x, dummies], axis=1)
k = df2['Price']
z=df2[['Range_sum', 'HP_sum', 'Chargetime_sum']] 

In [51]:
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')

In [52]:
model.first_stage

,Price
R-squared,0.6001
Partial R-squared,0.0678
Shea's R-squared,0.0678
Partial F-statistic,39.418
P-value (Partial F-stat),1.416e-08
Partial F-stat Distn,chi2(3)
==========================,===========
const,-552.26
,(-4.7676)
Range,0.8445


In [53]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -0.5898
Estimator:                    IV-2SLS   Adj. R-squared:                -0.6141
No. Observations:                 334   F-statistic:                    56.627
Date:                Mon, Apr 29 2024   P-value (F-stat)                0.0000
Time:                        13:05:50   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -5.1039     1.1351    -4.4963     0.0000     -7.3288     -2.8791
Range          0.1258     0.0235     5.3609     0.0000      0.0798      0.1718
HP             0.1203     0.0419     2.8693     0.0041      0.0381      0.2025
Chargetime    -0.0393     0.0162    -2.4310     0.0151     -0.0710     -0.0076
China         -3.4112     0.5742    -5.9412     0.0000     -4.5365     -2.2859
Price         -0.1380     0.0338    -4.0837     0.0000     -0.2042     -0.0717
==============================================================================

Endogenous: Price
Instruments: Range_sum, HP_sum, Chargetime_sum
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Gandhi Houde instrument (Quadratic diff)

In [54]:
z=df2[['Range_quadratic', 'HP_quadratic', 'Chargetime_quadratic']] 
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')
model.first_stage

,Price
R-squared,0.5962
Partial R-squared,0.0585
Shea's R-squared,0.0585
Partial F-statistic,24.511
P-value (Partial F-stat),1.954e-05
Partial F-stat Distn,chi2(3)
==========================,===========
const,-559.35
,(-4.6257)
Range,0.7943


In [55]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                     -0.6234
Estimator:                    IV-2SLS   Adj. R-squared:                -0.6482
No. Observations:                 334   F-statistic:                    46.196
Date:                Mon, Apr 29 2024   P-value (F-stat)                0.0000
Time:                        13:05:50   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -5.0764     1.1513    -4.4094     0.0000     -7.3328     -2.8200
Range          0.1267     0.0254     4.9848     0.0000      0.0769      0.1765
HP             0.1228     0.0456     2.6952     0.0070      0.0335      0.2121
Chargetime    -0.0398     0.0166    -2.4000     0.0164     -0.0724     -0.0073
China         -3.4359     0.6190    -5.5508     0.0000     -4.6491     -2.2227
Price         -0.1402     0.0380    -3.6840     0.0002     -0.2147     -0.0656
==============================================================================

Endogenous: Price
Instruments: Range_quadratic, HP_quadratic, Chargetime_quadratic
Robust Covariance (Heteroskedastic)
Debiased: False
"""

# Gandhi Houde instrument (Local Diff)

In [56]:
z=df2[['Range_instrument_localdiff', 'HP_instrument_localdiff', 'Chargetime_instrument_localdiff']] 
model = IV2SLS(dependent=y, exog=X, endog=k, instruments=z).fit(cov_type='robust')
model.first_stage

,Price
R-squared,0.5911
Partial R-squared,0.0468
Shea's R-squared,0.0468
Partial F-statistic,9.8961
P-value (Partial F-stat),0.0195
Partial F-stat Distn,chi2(3)
=================================,===========
const,-5.6265
,(-0.2761)
Range,0.5783


In [57]:
model.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:       log_market_share   R-squared:                      0.0568
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0425
No. Observations:                 334   F-statistic:                    64.457
Date:                Mon, Apr 29 2024   P-value (F-stat)                0.0000
Time:                        13:05:50   Distribution:                  chi2(5)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -5.8503     0.8972    -6.5205     0.0000     -7.6087     -4.0918
Range          0.1019     0.0176     5.7999     0.0000      0.0675      0.1363
HP             0.0532     0.0388     1.3708     0.1704     -0.0229      0.1293
Chargetime    -0.0245     0.0147    -1.6667     0.0956     -0.0534      0.0043
China         -2.7420     0.5328    -5.1461     0.0000     -3.7863     -1.6976
Price         -0.0782     0.0318    -2.4580     0.0140     -0.1405     -0.0158
==============================================================================

Endogenous: Price
Instruments: Range_instrument_localdiff, HP_instrument_localdiff, Chargetime_instrument_localdiff
Robust Covariance (Heteroskedastic)
Debiased: False
"""